In [33]:
import json
import pandas as pd

with open('data/sub-01/explicit_block1.json') as f:
    data = json.load(f)

df = pd.DataFrame(data[0])

In [34]:
df

,num_trial,iti_duration,modality,v_leading,v_trailing,v_pred,a_leading,a_trailing,a_pred,target,...,reaction_time,timeout,start_trial_absolute,start_fixation,start_leading,start_isi,start_trailing,start_response,end_trial,full_screen
0,0,1.807468,visual,90,45,EXP,None,None,None,None,...,0.491824,False,2025-04-02 13:55:11.625848,0.005268,1.807606,2.309602,2.807738,3.307744,3.808044,Yes
1,1,2.215387,visual,0,45,UEX,None,None,None,None,...,0.867087,False,2025-04-02 13:55:15.433899,0.000964,2.215523,2.717023,3.215756,3.715766,4.584075,Yes
2,2,1.906897,visual,90,135,UEX,None,None,None,None,...,2.207662,False,2025-04-02 13:55:20.017978,0.001284,1.907053,2.407978,2.907158,3.407163,5.615934,Yes
3,3,1.942440,visual,90,45,EXP,None,None,None,None,...,0.258076,False,2025-04-02 13:55:25.633918,0.000887,1.942819,2.445187,2.943108,3.443117,3.703980,Yes
4,4,1.821628,visual,90,135,UEX,None,None,None,None,...,1.028795,False,2025-04-02 13:55:29.337904,0.001124,1.821764,2.324144,2.821888,3.321892,4.352031,Yes
5,5,1.561327,visual,0,45,UEX,None,None,None,None,...,0.633120,False,2025-04-02 13:55:33.689941,0.000949,1.561464,2.062379,2.561681,3.061687,3.696019,Yes
6,6,1.973593,visual,90,135,UEX,None,None,None,None,...,0.588864,False,2025-04-02 13:55:37.385964,0.001055,1.973814,2.474716,2.974030,3.474036,4.064012,Yes
7,7,1.916262,visual,90,45,EXP,None,None,None,None,...,0.366127,False,2025-04-02 13:55:41.449985,0.000886,1.916503,2.417481,2.916770,3.416777,3.784004,Yes
8,8,1.854705,visual,0,135,EXP,None,None,None,None,...,0.482520,False,2025-04-02 13:55:45.233994,0.001269,1.854843,2.357103,2.855036,3.355040,3.839920,Yes
9,9,2.211486,visual,0,135,EXP,None,None,None,None,...,0.270663,False,2025-04-02 13:55:49.073918,0.000944,2.212063,2.713009,3.212175,3.712178,3.984042,Yes


In [36]:
df.groupby(["v_leading", "v_pred", "response"])["outcome"].value_counts()

v_leading  v_pred  response    outcome
0          EXP     frequent    1          1
                   infrequent  0          2
           UEX     frequent    0          2
                   infrequent  1          1
90         EXP     frequent    1          1
                   infrequent  0          2
           UEX     frequent    0          2
                   infrequent  1          1
Name: count, dtype: int64

In [6]:
with open('data/sub-04/sub-04_info.json') as f:
    data = json.load(f)

#df_info = pd.DataFrame(data)

In [15]:
df_learning = pd.DataFrame(data["conditions_learning_2"])
df_test = pd.DataFrame(data["conditions_test_3"])

In [16]:
df_learning.groupby("v_leading")["v_trailing"].value_counts()

v_leading  v_trailing
0          135           24
           45             8
90         45            24
           135            8
neutralV   45            16
           135           16
Name: count, dtype: int64

In [17]:
df_test.groupby("v_leading")["v_trailing"].value_counts()

v_leading  v_trailing
0          135           24
           45             8
90         45            24
           135            8
neutralV   45            16
           135           16
Name: count, dtype: int64

In [3]:
df[["v_pred","target", "response", "outcome", "visual_mapping"]]

,v_pred,target,response,outcome,visual_mapping
0,EXP,0,normal,1,1
1,EXP,1,deviant,1,1
2,UEX,0,normal,1,1
3,neutral,0,normal,1,1
4,EXP,0,deviant,0,1
5,neutral,0,deviant,0,1
6,EXP,1,deviant,1,1
7,EXP,1,deviant,1,1
8,EXP,0,normal,1,1
9,neutral,1,deviant,1,1
